## Description

Bone models of only summer 18S rRNA data at all levels, using the ml_new environment, rarefied data and no normalization during import, n = 1000 estimators, k fold = 6 (for regular, non-nested cross-validation), and hyperparameter tuning. Incorporates no metadata features.

Note: only went up to level 12 because levels 12-15 collapse at the exact same taxa.

In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing, svm, metrics
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV, GroupKFold, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_error, roc_auc_score, precision_score, make_scorer
from sklearn.linear_model import LassoCV, Lasso, LassoLarsIC, ElasticNet, LassoLarsCV
from sklearn.externals import joblib
from scipy.stats import randint as sp_randint
from matplotlib import pyplot as plt
from sklearn.feature_selection import RFECV
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier

/Users/heatherdeel/opt/miniconda3/envs/ml/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
%matplotlib inline
import biom
import calour as ca
from scipy import stats
import scipy
import pickle
import time
import math
import inspect
pd.set_option('display.max_rows', 10000)

/Users/heatherdeel/opt/miniconda3/envs/ml/lib/python3.7/site-packages/skbio/util/_testing.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as pdt


## Import data at the ASV level

In [3]:
exp_ASV = ca.read_amplicon('/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/01_qiime2_analysis/03_feature_tables/18S_plate52_ribonly_table_full_taxfiltered_214940.biom', '/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/02_metadata/map_18S.txt', min_reads=0, normalize=None)


2020-07-02 11:39:27 WARNING These have metadata but do not have data - dropped (59): {'STAFS.2016.065.L10', 'CSU.extraction.blank11', 'STAFS.2016.065.L11', 'BLANK52.9A', 'BLANK52.9H', 'CSU.extraction.blank1', 'BLANK52.9C', 'BLANK52.10D', 'BLANK52.12A', 'CSU.extraction.blank12', 'CSU.extraction.blank6', 'CSU.extraction.blank4', 'CSU.extraction.blank8', 'CSU.extraction.blank13', 'BLANK52.11D', 'BLANK52.12B', 'BLANK52.9G', 'CSU.extraction.blank10', 'CSU.extraction.blank2', 'BLANK52.11F', 'BLANK52.5B', 'BLANK52.10F', 'BLANK52.11C', 'CSU.extraction.blank14', 'BLANK52.10B', 'SHSU.STAFS2016.011.L09', 'BLANK52.9B', 'STAFS.2016.065.L09', 'BLANK52.10G', 'BLANK52.11A', 'BLANK52.12D', 'BLANK52.10E', 'SHSU.STAFS2016.024.L10', 'CSU.extraction.blank15', 'BLANK52.12F', 'BLANK52.9F', 'BLANK52.11B', 'STAFS.2016.064.R09', 'BLANK52.10A', 'SHSU.STAFS2016.007.L09', 'CSU.extraction.blank9', '#q2:types', 'SHSU.STAFS2016.007.R10', 'BLANK52.11E', 'BLANK52.12G', 'BLANK52.9D', 'BLANK52.11H', 'BLANK52.11G', 'BLANK

## Filter to only summer ASV data

In [4]:
#filtering out spring and defining this as "Summer"
Summer_ASV = exp_ASV.filter_samples('season', 'summer')
Summer_ASV.sample_metadata.host_subject_id.value_counts()

STAFS2016.064    7
STAFS2016.067    6
STAFS2016.065    5
Name: host_subject_id, dtype: int64

In [5]:
Summer_ASV.sample_metadata['ADD']

SampleID
STAFS.2016.064.L09         4013.333333
STAFS.2016.065.R08         4013.333333
STAFS.2016.067.L10         3623.055556
STAFS.2016.067.R10                1645
STAFS.2016.064.R12         2035.277778
STAFS.2016.067.R09         1877.777778
STAFS.2016.065.R10         2804.722222
STAFS.2016.067.L12         4366.111111
STAFS.2016.065.R09         3456.111111
STAFS.2016.064.R10         3456.111111
STAFS.2016.065.L12         2268.055556
STAFS.2016.064.L10         4756.388889
STAFS.2016.065.R11         4756.388889
STAFS.2016.067.R11         1151.666667
STAFS.2016.064.L12         2268.055556
STAFS.2016.067.L11.june    5201.388889
STAFS.2016.064.R11         1300.555556
STAFS.2016.064.L11         2804.722222
Name: ADD, dtype: object

In [6]:
print(Summer_ASV.feature_metadata)

                                                       _feature_id
4bdd78c3c45acd439bdd1881ea7eaac2  4bdd78c3c45acd439bdd1881ea7eaac2
a05b1c9560df7eceb2d90e8b6f5ef2aa  a05b1c9560df7eceb2d90e8b6f5ef2aa
a0c745282e0f7e513c095ec030e80780  a0c745282e0f7e513c095ec030e80780
9e048aa0ed66db3f1c3d3d4063ac22e3  9e048aa0ed66db3f1c3d3d4063ac22e3
1feb171007495e3e1d9994306a77d004  1feb171007495e3e1d9994306a77d004
2466b3ba732737511076437584b5bc9f  2466b3ba732737511076437584b5bc9f
df6ef6f198417d48e8bbbca8d9353d1e  df6ef6f198417d48e8bbbca8d9353d1e
9274fbc42446bdfd64b443e4b16fd168  9274fbc42446bdfd64b443e4b16fd168
8fe031ef4a665ed43b47b0ded2101c7b  8fe031ef4a665ed43b47b0ded2101c7b
014d1667ebbccd7d439d488cdb01b05f  014d1667ebbccd7d439d488cdb01b05f
e38ec0a2f0070193558338feac87c976  e38ec0a2f0070193558338feac87c976
b5d33326dce0f687283a8ceee7714404  b5d33326dce0f687283a8ceee7714404
3526664f852de7e298943fe195ffa86e  3526664f852de7e298943fe195ffa86e
39640d42a246ac16106c367552162ac4  39640d42a246ac16106c36755216

## Running the Summer ASV Model

In [7]:
# groupKfold = 3, will leave out one body for each model in spring data
gkf = GroupKFold(3)

X = Summer_ASV.data
y = Summer_ASV.sample_metadata['ADD']
y = (y.astype(float))

groups = Summer_ASV.sample_metadata['host_subject_id']

# used to test the param grid for parameter tuning
# use the output of this estimator below for input into new estimator (not commented out)
param_grid = {"max_depth": [4, 8, 16, None],
              "max_features": ['sqrt', 'log2', 0.1],
              "min_samples_split": [0.001, 0.01, 0.1],
              "min_weight_fraction_leaf": [0.0001, 0.001, 0.01],
              "bootstrap": [True, False]}

#param_grid = {"max_depth": [8],
#          "max_features": [0.1],
#          "min_samples_split": [0.001],
#          "min_weight_fraction_leaf": [0.0001],
#          "bootstrap": [False]}

rf = RandomForestRegressor(n_estimators=1000, random_state=999, criterion='mae')
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=1)

In [8]:
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x11f18cf50>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mae', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_es...
                                             oob_score=False, random_state=999,
                                             verbose=0, warm_start=False

In [9]:
# this line is used when run with the first param grid to determine what the best parameters are for tuning
print(gs.best_params_)

{'bootstrap': False, 'max_depth': 8, 'max_features': 0.1, 'min_samples_split': 0.001, 'min_weight_fraction_leaf': 0.0001}


In [10]:
print('The best mean absolute error is', round(-gs.best_score_,2))

The best mean absolute error is 990.91


In [11]:
joblib.dump(gs.best_estimator_, 'models/bone_summer_18S_ASV.pkl')

['models/bone_summer_18S_ASV.pkl']

In [12]:
bone_summer_ASV = joblib.load('models/bone_summer_18S_ASV.pkl')
bone_summer_ASV

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=8, max_features=0.1, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.001, min_weight_fraction_leaf=0.0001,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

## Determine important features of the ASV model

In [14]:
importances = bone_summer_ASV.feature_importances_
std = np.std([tree.feature_importances_ for tree in bone_summer_ASV.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

feature_metadata = Summer_ASV.feature_metadata
np.savetxt("importances/bone_summer_18S_ASV_features.csv", feature_metadata, delimiter=",", fmt='%s')


#print the important ids in order
general_importances = []
count = 0
print("Feature:\t\t\t\tImportance:")
for i in indices:
    general_importances += (Summer_ASV.feature_metadata.index.values[i], importances[indices[count]])
    if count < 25:
        print(str(count+1)+". "+str(Summer_ASV.feature_metadata.index.values[i])+"\t"+str(importances[indices[count]]))
    count += 1
    
general_importances_df = pd.DataFrame(np.array(general_importances).reshape(2864,1))

np.savetxt("importances/bone_summer_18S_ASV_importances.csv", general_importances_df, delimiter=",", fmt='%s')

Feature:				Importance:
1. c020065c9c2b9fdd84566b334aad62cf	0.05480054453272294
2. 0645707bddd580dc97b168d33129fa91	0.05157634246094172
3. 86615a5aeabc0f233f61aaa2c25187a4	0.04840308762488264
4. 9c66c621dda783dfb06b481cfcf134c4	0.04136826274665938
5. 36b5a020f6259803c30cc567ed748b8d	0.03759905061777705
6. 174b236c9025192e4973d425cdda9aa0	0.03405721942420768
7. abcf0b825ba29dbc839570d227922ca3	0.021738478423398433
8. 9e3fb34da3dc672a429d6f54365734c5	0.021479394451284516
9. bf808221c79c379a6a2ea02289939774	0.019792077086550248
10. f86e178e49cd121fcc9f59e48c1fc091	0.018327807158578333
11. c62962af8e338d4b5da35b4ac5b40eb7	0.01675117959766088
12. 586e64dbe408b5ffff04fc660f45ef14	0.016403415487373566
13. f827d14fa967952a51d25b3e347e763f	0.015000042764413909
14. fde13df564f0250bc420ee2f1833fb7b	0.014921982582416062
15. 4ce65a8b000737c8bd9a28ec392e7aeb	0.012401776168298843
16. b032b792bb610a5f44715c3bb2b366fd	0.011813644855285486
17. d4cf65c9d5ef51575fb31825f8bede24	0.011681643881635818
18. 02

## Import rarefied data collapsed at level 12

In [15]:
exp_L12 = ca.read_amplicon('/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/01_qiime2_analysis/03_feature_tables/collapsed_tables/18S_plate52_ribonly_table_full_taxfiltered_214940_L12.biom', '/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/02_metadata/map_18S.txt', min_reads=0, normalize=None)


2020-07-02 11:58:54 WARNING These have metadata but do not have data - dropped (59): {'STAFS.2016.065.L10', 'CSU.extraction.blank11', 'STAFS.2016.065.L11', 'BLANK52.9A', 'BLANK52.9H', 'CSU.extraction.blank1', 'BLANK52.9C', 'BLANK52.10D', 'BLANK52.12A', 'CSU.extraction.blank12', 'CSU.extraction.blank6', 'CSU.extraction.blank4', 'CSU.extraction.blank8', 'CSU.extraction.blank13', 'BLANK52.11D', 'BLANK52.12B', 'BLANK52.9G', 'CSU.extraction.blank10', 'CSU.extraction.blank2', 'BLANK52.11F', 'BLANK52.5B', 'BLANK52.10F', 'BLANK52.11C', 'CSU.extraction.blank14', 'BLANK52.10B', 'SHSU.STAFS2016.011.L09', 'BLANK52.9B', 'STAFS.2016.065.L09', 'BLANK52.10G', 'BLANK52.11A', 'BLANK52.12D', 'BLANK52.10E', 'SHSU.STAFS2016.024.L10', 'CSU.extraction.blank15', 'BLANK52.12F', 'BLANK52.9F', 'BLANK52.11B', 'STAFS.2016.064.R09', 'BLANK52.10A', 'SHSU.STAFS2016.007.L09', 'CSU.extraction.blank9', '#q2:types', 'SHSU.STAFS2016.007.R10', 'BLANK52.11E', 'BLANK52.12G', 'BLANK52.9D', 'BLANK52.11H', 'BLANK52.11G', 'BLANK

In [16]:
#filtering out spring and defining this as "Summer"
Summer_L12 = exp_L12.filter_samples('season', 'summer')
Summer_L12.sample_metadata.host_subject_id.value_counts()

STAFS2016.064    7
STAFS2016.067    6
STAFS2016.065    5
Name: host_subject_id, dtype: int64

In [17]:
print(Summer_L12.feature_metadata)

                                                                                          _feature_id
D_0__Eukaryota;D_1__Amoebozoa;D_2__Cavosteliida...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Cavosteliid...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Dictyostelia...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Dictyosteli...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;

## Running the L12 model

In [18]:
# groupKfold = 3, will leave out one body for each model
gkf = GroupKFold(3)

X = Summer_L12.data
y = Summer_L12.sample_metadata['ADD']
y = (y.astype(float))

groups = Summer_L12.sample_metadata['host_subject_id']

# used to test the param grid for parameter tuning
# use the output of this estimator below for input into new estimator (not commented out)
param_grid = {"max_depth": [4, 8, 16, None],
              "max_features": ['sqrt', 'log2', 0.1],
              "min_samples_split": [0.001, 0.01, 0.1],
              "min_weight_fraction_leaf": [0.0001, 0.001, 0.01],
              "bootstrap": [True, False]}

#param_grid = {"max_depth": [8],
#          "max_features": [0.1],
#          "min_samples_split": [0.001],
#          "min_weight_fraction_leaf": [0.0001],
#          "bootstrap": [False]}

rf = RandomForestRegressor(n_estimators=1000, random_state=999, criterion='mae')
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=1)

In [19]:
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x120ec7750>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mae', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_es...
                                             oob_score=False, random_state=999,
                                             verbose=0, warm_start=False

In [20]:
# this line is used when run with the first param grid to determine what the best parameters are for tuning
print(gs.best_params_)

{'bootstrap': False, 'max_depth': 4, 'max_features': 0.1, 'min_samples_split': 0.001, 'min_weight_fraction_leaf': 0.0001}


In [21]:
print('The best mean absolute error is', round(-gs.best_score_,2))

The best mean absolute error is 877.59


In [22]:
joblib.dump(gs.best_estimator_, 'models/bone_summer_18S_L12.pkl')

['models/bone_summer_18S_L12.pkl']

In [23]:
bone_summer_L12 = joblib.load('models/bone_summer_18S_L12.pkl')
bone_summer_L12

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=4, max_features=0.1, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.001, min_weight_fraction_leaf=0.0001,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

## Determine important features of the summer L12 model

In [24]:
importances = bone_summer_L12.feature_importances_
std = np.std([tree.feature_importances_ for tree in bone_summer_L12.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

feature_metadata = Summer_L12.feature_metadata
np.savetxt("importances/bone_summer_18S_L12_features.csv", feature_metadata, delimiter=",", fmt='%s')


#print the important ids in order
general_importances = []
count = 0
print("Feature:\t\t\t\tImportance:")
for i in indices:
    general_importances += (Summer_L12.feature_metadata.index.values[i], importances[indices[count]])
    if count < 25:
        print(str(count+1)+". "+str(Summer_L12.feature_metadata.index.values[i])+"\t"+str(importances[indices[count]]))
    count += 1
    
general_importances_df = pd.DataFrame(np.array(general_importances).reshape(664,1))

np.savetxt("importances/bone_summer_18S_L12_importances.csv", general_importances_df, delimiter=",", fmt='%s')

Feature:				Importance:
1. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Ascomycota;D_6__Saccharomycotina;D_7__Saccharomycetes;D_8__Saccharomycetales;D_9__Dipodascaceae;D_10__Yarrowia;D_11__Yarrowia lipolytica	0.06689861305175253
2. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Ascomycota;D_6__Pezizomycotina;D_7__Eurotiomycetes;__;__;__;__	0.05596759443643954
3. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Ascomycota;D_6__Saccharomycotina;D_7__Saccharomycetes;D_8__Saccharomycetales;D_9__Incertae Sedis;D_10__Starmerella-Candida clade;D_11__[Candida] bombi	0.05555132525027306
4. D_0__Eukaryota;D_1__Amoebozoa;D_2__Tubulinea;D_3__Euamoebida;D_4__BOLA868;D_5__metagenome;D_6__;D_7__;D_8__;D_9__;D_10__;D_11__	0.03828653024501245
5. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Ascomycota;D_6__Pezizomycotina;D_7__Sordariomycetes;D_8__Hypocreales;__;__;__	0.032424

## Import rarefied data collapsed at level 11

In [25]:
exp_L11 = ca.read_amplicon('/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/01_qiime2_analysis/03_feature_tables/collapsed_tables/18S_plate52_ribonly_table_full_taxfiltered_214940_L11.biom', '/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/02_metadata/map_18S.txt', min_reads=0, normalize=None)


2020-07-02 12:20:56 WARNING These have metadata but do not have data - dropped (59): {'STAFS.2016.065.L10', 'CSU.extraction.blank11', 'STAFS.2016.065.L11', 'BLANK52.9A', 'BLANK52.9H', 'CSU.extraction.blank1', 'BLANK52.9C', 'BLANK52.10D', 'BLANK52.12A', 'CSU.extraction.blank12', 'CSU.extraction.blank6', 'CSU.extraction.blank4', 'CSU.extraction.blank8', 'CSU.extraction.blank13', 'BLANK52.11D', 'BLANK52.12B', 'BLANK52.9G', 'CSU.extraction.blank10', 'CSU.extraction.blank2', 'BLANK52.11F', 'BLANK52.5B', 'BLANK52.10F', 'BLANK52.11C', 'CSU.extraction.blank14', 'BLANK52.10B', 'SHSU.STAFS2016.011.L09', 'BLANK52.9B', 'STAFS.2016.065.L09', 'BLANK52.10G', 'BLANK52.11A', 'BLANK52.12D', 'BLANK52.10E', 'SHSU.STAFS2016.024.L10', 'CSU.extraction.blank15', 'BLANK52.12F', 'BLANK52.9F', 'BLANK52.11B', 'STAFS.2016.064.R09', 'BLANK52.10A', 'SHSU.STAFS2016.007.L09', 'CSU.extraction.blank9', '#q2:types', 'SHSU.STAFS2016.007.R10', 'BLANK52.11E', 'BLANK52.12G', 'BLANK52.9D', 'BLANK52.11H', 'BLANK52.11G', 'BLANK

In [26]:
#filtering out spring and defining this as "Summer"
Summer_L11 = exp_L11.filter_samples('season', 'summer')
Summer_L11.sample_metadata.host_subject_id.value_counts()

STAFS2016.064    7
STAFS2016.067    6
STAFS2016.065    5
Name: host_subject_id, dtype: int64

In [27]:
print(Summer_L11.feature_metadata)

                                                                                          _feature_id
D_0__Eukaryota;D_1__Amoebozoa;D_2__Cavosteliida...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Cavosteliid...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Dictyostelia...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Dictyosteli...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;

## Running the L11 model

In [28]:
# groupKfold = 3, will leave out one body for each model
gkf = GroupKFold(3)

X = Summer_L11.data
y = Summer_L11.sample_metadata['ADD']
y = (y.astype(float))

groups = Summer_L11.sample_metadata['host_subject_id']

# used to test the param grid for parameter tuning
# use the output of this estimator below for input into new estimator (not commented out)
param_grid = {"max_depth": [4, 8, 16, None],
              "max_features": ['sqrt', 'log2', 0.1],
              "min_samples_split": [0.001, 0.01, 0.1],
              "min_weight_fraction_leaf": [0.0001, 0.001, 0.01],
              "bootstrap": [True, False]}

#param_grid = {"max_depth": [8],
#          "max_features": [0.1],
#          "min_samples_split": [0.001],
#          "min_weight_fraction_leaf": [0.0001],
#          "bootstrap": [False]}

rf = RandomForestRegressor(n_estimators=1000, random_state=999, criterion='mae')
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=1)

In [29]:
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x11f4f6ad0>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mae', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_es...
                                             oob_score=False, random_state=999,
                                             verbose=0, warm_start=False

In [30]:
# this line is used when run with the first param grid to determine what the best parameters are for tuning
print(gs.best_params_)

{'bootstrap': False, 'max_depth': 4, 'max_features': 0.1, 'min_samples_split': 0.001, 'min_weight_fraction_leaf': 0.0001}


In [31]:
print('The best mean absolute error is', round(-gs.best_score_,2))

The best mean absolute error is 864.45


In [32]:
joblib.dump(gs.best_estimator_, 'models/bone_summer_18S_L11.pkl')

['models/bone_summer_18S_L11.pkl']

In [33]:
bone_summer_L11 = joblib.load('models/bone_summer_18S_L11.pkl')
bone_summer_L11

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=4, max_features=0.1, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.001, min_weight_fraction_leaf=0.0001,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

## Determine important features of the L11 model

In [34]:
importances = bone_summer_L11.feature_importances_
std = np.std([tree.feature_importances_ for tree in bone_summer_L11.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

feature_metadata = Summer_L11.feature_metadata
np.savetxt("importances/bone_summer_18S_L11_features.csv", feature_metadata, delimiter=",", fmt='%s')


#print the important ids in order
general_importances = []
count = 0
print("Feature:\t\t\t\tImportance:")
for i in indices:
    general_importances += (Summer_L11.feature_metadata.index.values[i], importances[indices[count]])
    if count < 25:
        print(str(count+1)+". "+str(Summer_L11.feature_metadata.index.values[i])+"\t"+str(importances[indices[count]]))
    count += 1
    
general_importances_df = pd.DataFrame(np.array(general_importances).reshape(634,1))

np.savetxt("importances/bone_summer_18S_L11_importances.csv", general_importances_df, delimiter=",", fmt='%s')

Feature:				Importance:
1. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Ascomycota;D_6__Saccharomycotina;D_7__Saccharomycetes;D_8__Saccharomycetales;D_9__Dipodascaceae;D_10__Yarrowia	0.06438777325683644
2. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Ascomycota;D_6__Pezizomycotina;D_7__Eurotiomycetes;__;__;__	0.05917676348909162
3. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Ascomycota;D_6__Saccharomycotina;D_7__Saccharomycetes;D_8__Saccharomycetales;D_9__Incertae Sedis;D_10__Starmerella-Candida clade	0.05481235127387432
4. D_0__Eukaryota;D_1__Amoebozoa;D_2__Tubulinea;D_3__Euamoebida;D_4__BOLA868;D_5__metagenome;D_6__;D_7__;D_8__;D_9__;D_10__	0.03688595348212524
5. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Ascomycota;D_6__Pezizomycotina;D_7__Sordariomycetes;D_8__Hypocreales;__;__	0.03434810163319503
6. D_0__Eukaryota;D_1__Opisthokonta;D_2__Holozoa;

## Import rarefied data collapsed at level 10

In [35]:
exp_L10 = ca.read_amplicon('/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/01_qiime2_analysis/03_feature_tables/collapsed_tables/18S_plate52_ribonly_table_full_taxfiltered_214940_L10.biom', '/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/02_metadata/map_18S.txt', min_reads=0, normalize=None)


2020-07-02 13:26:44 WARNING These have metadata but do not have data - dropped (59): {'STAFS.2016.065.L10', 'CSU.extraction.blank11', 'STAFS.2016.065.L11', 'BLANK52.9A', 'BLANK52.9H', 'CSU.extraction.blank1', 'BLANK52.9C', 'BLANK52.10D', 'BLANK52.12A', 'CSU.extraction.blank12', 'CSU.extraction.blank6', 'CSU.extraction.blank4', 'CSU.extraction.blank8', 'CSU.extraction.blank13', 'BLANK52.11D', 'BLANK52.12B', 'BLANK52.9G', 'CSU.extraction.blank10', 'CSU.extraction.blank2', 'BLANK52.11F', 'BLANK52.5B', 'BLANK52.10F', 'BLANK52.11C', 'CSU.extraction.blank14', 'BLANK52.10B', 'SHSU.STAFS2016.011.L09', 'BLANK52.9B', 'STAFS.2016.065.L09', 'BLANK52.10G', 'BLANK52.11A', 'BLANK52.12D', 'BLANK52.10E', 'SHSU.STAFS2016.024.L10', 'CSU.extraction.blank15', 'BLANK52.12F', 'BLANK52.9F', 'BLANK52.11B', 'STAFS.2016.064.R09', 'BLANK52.10A', 'SHSU.STAFS2016.007.L09', 'CSU.extraction.blank9', '#q2:types', 'SHSU.STAFS2016.007.R10', 'BLANK52.11E', 'BLANK52.12G', 'BLANK52.9D', 'BLANK52.11H', 'BLANK52.11G', 'BLANK

In [36]:
#filtering out spring and defining this as "Summer"
Summer_L10 = exp_L10.filter_samples('season', 'summer')
Summer_L10.sample_metadata.host_subject_id.value_counts()

STAFS2016.064    7
STAFS2016.067    6
STAFS2016.065    5
Name: host_subject_id, dtype: int64

In [37]:
print(Summer_L10.feature_metadata)

                                                                                          _feature_id
D_0__Eukaryota;D_1__Amoebozoa;D_2__Cavosteliida...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Cavosteliid...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Dictyostelia...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Dictyosteli...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;

## Running the L10 model

In [38]:
# groupKfold = 3, will leave out one body for each model
gkf = GroupKFold(3)

X = Summer_L10.data
y = Summer_L10.sample_metadata['ADD']
y = (y.astype(float))

groups = Summer_L10.sample_metadata['host_subject_id']

# used to test the param grid for parameter tuning
# use the output of this estimator below for input into new estimator (not commented out)
param_grid = {"max_depth": [4, 8, 16, None],
              "max_features": ['sqrt', 'log2', 0.1],
              "min_samples_split": [0.001, 0.01, 0.1],
              "min_weight_fraction_leaf": [0.0001, 0.001, 0.01],
              "bootstrap": [True, False]}

#param_grid = {"max_depth": [8],
#          "max_features": [0.1],
#          "min_samples_split": [0.001],
#          "min_weight_fraction_leaf": [0.0001],
#          "bootstrap": [False]}

rf = RandomForestRegressor(n_estimators=1000, random_state=999, criterion='mae')
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=1)

In [39]:
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x11f92f9d0>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mae', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_es...
                                             oob_score=False, random_state=999,
                                             verbose=0, warm_start=False

In [40]:
# this line is used when run with the first param grid to determine what the best parameters are for tuning
print(gs.best_params_)

{'bootstrap': False, 'max_depth': 4, 'max_features': 0.1, 'min_samples_split': 0.001, 'min_weight_fraction_leaf': 0.0001}


In [41]:
print('The best mean absolute error is', round(-gs.best_score_,2))

The best mean absolute error is 858.58


In [42]:
joblib.dump(gs.best_estimator_, 'models/bone_summer_18S_L10.pkl')

['models/bone_summer_18S_L10.pkl']

In [43]:
bone_summer_L10 = joblib.load('models/bone_summer_18S_L10.pkl')
bone_summer_L10

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=4, max_features=0.1, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.001, min_weight_fraction_leaf=0.0001,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

## Determine important features of the L10 model

In [44]:
importances = bone_summer_L10.feature_importances_
std = np.std([tree.feature_importances_ for tree in bone_summer_L10.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

feature_metadata = Summer_L10.feature_metadata
np.savetxt("importances/bone_summer_18S_L10_features.csv", feature_metadata, delimiter=",", fmt='%s')


#print the important ids in order
general_importances = []
count = 0
print("Feature:\t\t\t\tImportance:")
for i in indices:
    general_importances += (Summer_L10.feature_metadata.index.values[i], importances[indices[count]])
    if count < 25:
        print(str(count+1)+". "+str(Summer_L10.feature_metadata.index.values[i])+"\t"+str(importances[indices[count]]))
    count += 1
    
general_importances_df = pd.DataFrame(np.array(general_importances).reshape(594,1))

np.savetxt("importances/bone_summer_18S_L10_importances.csv", general_importances_df, delimiter=",", fmt='%s')

Feature:				Importance:
1. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Ascomycota;D_6__Saccharomycotina;D_7__Saccharomycetes;D_8__Saccharomycetales;D_9__Dipodascaceae	0.06434223212290009
2. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Ascomycota;D_6__Pezizomycotina;D_7__Eurotiomycetes;__;__	0.060845333047361796
3. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Ascomycota;D_6__Saccharomycotina;D_7__Saccharomycetes;D_8__Saccharomycetales;D_9__Incertae Sedis	0.05255468360528245
4. D_0__Eukaryota;D_1__Amoebozoa;D_2__Tubulinea;D_3__Euamoebida;D_4__BOLA868;D_5__metagenome;D_6__;D_7__;D_8__;D_9__	0.04079168721325325
5. D_0__Eukaryota;D_1__Opisthokonta;D_2__Holozoa;D_3__Metazoa (Animalia);D_4__Eumetazoa;D_5__Bilateria;D_6__Nematoda;D_7__Chromadorea;D_8__Rhabditida;D_9__Oscheius tipulae	0.03236630068897268
6. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Ascomyco

## Import rarefied data collapsed at level 9

In [45]:
exp_L9 = ca.read_amplicon('/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/01_qiime2_analysis/03_feature_tables/collapsed_tables/18S_plate52_ribonly_table_full_taxfiltered_214940_L9.biom', '/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/02_metadata/map_18S.txt', min_reads=0, normalize=None)


2020-07-02 13:40:01 WARNING These have metadata but do not have data - dropped (59): {'STAFS.2016.065.L10', 'CSU.extraction.blank11', 'STAFS.2016.065.L11', 'BLANK52.9A', 'BLANK52.9H', 'CSU.extraction.blank1', 'BLANK52.9C', 'BLANK52.10D', 'BLANK52.12A', 'CSU.extraction.blank12', 'CSU.extraction.blank6', 'CSU.extraction.blank4', 'CSU.extraction.blank8', 'CSU.extraction.blank13', 'BLANK52.11D', 'BLANK52.12B', 'BLANK52.9G', 'CSU.extraction.blank10', 'CSU.extraction.blank2', 'BLANK52.11F', 'BLANK52.5B', 'BLANK52.10F', 'BLANK52.11C', 'CSU.extraction.blank14', 'BLANK52.10B', 'SHSU.STAFS2016.011.L09', 'BLANK52.9B', 'STAFS.2016.065.L09', 'BLANK52.10G', 'BLANK52.11A', 'BLANK52.12D', 'BLANK52.10E', 'SHSU.STAFS2016.024.L10', 'CSU.extraction.blank15', 'BLANK52.12F', 'BLANK52.9F', 'BLANK52.11B', 'STAFS.2016.064.R09', 'BLANK52.10A', 'SHSU.STAFS2016.007.L09', 'CSU.extraction.blank9', '#q2:types', 'SHSU.STAFS2016.007.R10', 'BLANK52.11E', 'BLANK52.12G', 'BLANK52.9D', 'BLANK52.11H', 'BLANK52.11G', 'BLANK

In [46]:
#filtering out spring and defining this as "Summer"
Summer_L9 = exp_L9.filter_samples('season', 'summer')
Summer_L9.sample_metadata.host_subject_id.value_counts()

STAFS2016.064    7
STAFS2016.067    6
STAFS2016.065    5
Name: host_subject_id, dtype: int64

In [47]:
print(Summer_L9.feature_metadata)

                                                                                          _feature_id
D_0__Eukaryota;D_1__Amoebozoa;D_2__Cavosteliida...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Cavosteliid...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Dictyostelia...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Dictyosteli...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;

## Running the L9 model

In [48]:
# groupKfold = 3, will leave out one body for each model
gkf = GroupKFold(3)

X = Summer_L9.data
y = Summer_L9.sample_metadata['ADD']
y = (y.astype(float))

groups = Summer_L9.sample_metadata['host_subject_id']

# used to test the param grid for parameter tuning
# use the output of this estimator below for input into new estimator (not commented out)
param_grid = {"max_depth": [4, 8, 16, None],
              "max_features": ['sqrt', 'log2', 0.1],
              "min_samples_split": [0.001, 0.01, 0.1],
              "min_weight_fraction_leaf": [0.0001, 0.001, 0.01],
              "bootstrap": [True, False]}

#param_grid = {"max_depth": [8],
#          "max_features": [0.1],
#          "min_samples_split": [0.001],
#          "min_weight_fraction_leaf": [0.0001],
#          "bootstrap": [False]}

rf = RandomForestRegressor(n_estimators=1000, random_state=999, criterion='mae')
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=1)

In [49]:
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x120ef69d0>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mae', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_es...
                                             oob_score=False, random_state=999,
                                             verbose=0, warm_start=False

In [50]:
# this line is used when run with the first param grid to determine what the best parameters are for tuning
print(gs.best_params_)

{'bootstrap': False, 'max_depth': 8, 'max_features': 0.1, 'min_samples_split': 0.001, 'min_weight_fraction_leaf': 0.0001}


In [51]:
print('The best mean absolute error is', round(-gs.best_score_,2))

The best mean absolute error is 870.4


In [52]:
joblib.dump(gs.best_estimator_, 'models/bone_summer_18S_L9.pkl')

['models/bone_summer_18S_L9.pkl']

In [53]:
bone_summer_L9 = joblib.load('models/bone_summer_18S_L9.pkl')
bone_summer_L9

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=8, max_features=0.1, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.001, min_weight_fraction_leaf=0.0001,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

## Determine important features of the L9 model

In [54]:
importances = bone_summer_L9.feature_importances_
std = np.std([tree.feature_importances_ for tree in bone_summer_L9.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

feature_metadata = Summer_L9.feature_metadata
np.savetxt("importances/bone_summer_18S_L9_features.csv", feature_metadata, delimiter=",", fmt='%s')


#print the important ids in order
general_importances = []
count = 0
print("Feature:\t\t\t\tImportance:")
for i in indices:
    general_importances += (Summer_L9.feature_metadata.index.values[i], importances[indices[count]])
    if count < 25:
        print(str(count+1)+". "+str(Summer_L9.feature_metadata.index.values[i])+"\t"+str(importances[indices[count]]))
    count += 1
    
general_importances_df = pd.DataFrame(np.array(general_importances).reshape(490,1))

np.savetxt("importances/bone_summer_18S_L9_importances.csv", general_importances_df, delimiter=",", fmt='%s')

Feature:				Importance:
1. D_0__Eukaryota;D_1__Opisthokonta;D_2__Holozoa;D_3__Metazoa (Animalia);D_4__Eumetazoa;D_5__Bilateria;D_6__Nematoda;D_7__Chromadorea;D_8__Rhabditida	0.06172926971207485
2. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Ascomycota;D_6__Saccharomycotina;D_7__Saccharomycetes;D_8__Saccharomycetales	0.06013836074512696
3. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Ascomycota;D_6__Pezizomycotina;D_7__Eurotiomycetes;__	0.05280307965602406
4. D_0__Eukaryota;D_1__Amoebozoa;D_2__Tubulinea;D_3__Euamoebida;D_4__BOLA868;D_5__metagenome;D_6__;D_7__;D_8__	0.041954084756124534
5. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Ascomycota;D_6__Pezizomycotina;D_7__Sordariomycetes;D_8__Hypocreales	0.03177142152465884
6. D_0__Eukaryota;D_1__SAR;D_2__Alveolata;__;__;__;__;__;__	0.02824800068484186
7. D_0__Eukaryota;D_1__SAR;D_2__Rhizaria;D_3__Cercozoa;D_4__Glissomonadida;D_5__Heteromi

## Import rarefied data collapsed at level 8

In [55]:
exp_L8 = ca.read_amplicon('/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/01_qiime2_analysis/03_feature_tables/collapsed_tables/18S_plate52_ribonly_table_full_taxfiltered_214940_L8.biom', '/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/02_metadata/map_18S.txt', min_reads=0, normalize=None)


2020-07-02 13:56:06 WARNING These have metadata but do not have data - dropped (59): {'STAFS.2016.065.L10', 'CSU.extraction.blank11', 'STAFS.2016.065.L11', 'BLANK52.9A', 'BLANK52.9H', 'CSU.extraction.blank1', 'BLANK52.9C', 'BLANK52.10D', 'BLANK52.12A', 'CSU.extraction.blank12', 'CSU.extraction.blank6', 'CSU.extraction.blank4', 'CSU.extraction.blank8', 'CSU.extraction.blank13', 'BLANK52.11D', 'BLANK52.12B', 'BLANK52.9G', 'CSU.extraction.blank10', 'CSU.extraction.blank2', 'BLANK52.11F', 'BLANK52.5B', 'BLANK52.10F', 'BLANK52.11C', 'CSU.extraction.blank14', 'BLANK52.10B', 'SHSU.STAFS2016.011.L09', 'BLANK52.9B', 'STAFS.2016.065.L09', 'BLANK52.10G', 'BLANK52.11A', 'BLANK52.12D', 'BLANK52.10E', 'SHSU.STAFS2016.024.L10', 'CSU.extraction.blank15', 'BLANK52.12F', 'BLANK52.9F', 'BLANK52.11B', 'STAFS.2016.064.R09', 'BLANK52.10A', 'SHSU.STAFS2016.007.L09', 'CSU.extraction.blank9', '#q2:types', 'SHSU.STAFS2016.007.R10', 'BLANK52.11E', 'BLANK52.12G', 'BLANK52.9D', 'BLANK52.11H', 'BLANK52.11G', 'BLANK

In [56]:
#filtering out spring and defining this as "Summer"
Summer_L8 = exp_L8.filter_samples('season', 'summer')
Summer_L8.sample_metadata.host_subject_id.value_counts()

STAFS2016.064    7
STAFS2016.067    6
STAFS2016.065    5
Name: host_subject_id, dtype: int64

In [57]:
print(Summer_L8.feature_metadata)

                                                                                          _feature_id
D_0__Eukaryota;D_1__Amoebozoa;D_2__Cavosteliida...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Cavosteliid...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Dictyostelia...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Dictyosteli...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;

## Running the L8 model

In [58]:
# groupKfold = 3, will leave out one body for each model
gkf = GroupKFold(3)

X = Summer_L8.data
y = Summer_L8.sample_metadata['ADD']
y = (y.astype(float))

groups = Summer_L8.sample_metadata['host_subject_id']

# used to test the param grid for parameter tuning
# use the output of this estimator below for input into new estimator (not commented out)
param_grid = {"max_depth": [4, 8, 16, None],
              "max_features": ['sqrt', 'log2', 0.1],
              "min_samples_split": [0.001, 0.01, 0.1],
              "min_weight_fraction_leaf": [0.0001, 0.001, 0.01],
              "bootstrap": [True, False]}

#param_grid = {"max_depth": [8],
#          "max_features": [0.1],
#          "min_samples_split": [0.001],
#          "min_weight_fraction_leaf": [0.0001],
#          "bootstrap": [False]}

rf = RandomForestRegressor(n_estimators=1000, random_state=999, criterion='mae')
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=1)

In [59]:
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x11f9a1a50>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mae', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_es...
                                             oob_score=False, random_state=999,
                                             verbose=0, warm_start=False

In [60]:
# this line is used when run with the first param grid to determine what the best parameters are for tuning
print(gs.best_params_)

{'bootstrap': False, 'max_depth': 4, 'max_features': 0.1, 'min_samples_split': 0.001, 'min_weight_fraction_leaf': 0.0001}


In [61]:
print('The best mean absolute error is', round(-gs.best_score_,2))

The best mean absolute error is 886.16


In [62]:
joblib.dump(gs.best_estimator_, 'models/bone_summer_18S_L8.pkl')

['models/bone_summer_18S_L8.pkl']

In [63]:
bone_summer_L8 = joblib.load('models/bone_summer_18S_L8.pkl')
bone_summer_L8

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=4, max_features=0.1, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.001, min_weight_fraction_leaf=0.0001,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

## Determine important features of the L8 model

In [64]:
importances = bone_summer_L8.feature_importances_
std = np.std([tree.feature_importances_ for tree in bone_summer_L8.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

feature_metadata = Summer_L8.feature_metadata
np.savetxt("importances/bone_summer_18S_L8_features.csv", feature_metadata, delimiter=",", fmt='%s')


#print the important ids in order
general_importances = []
count = 0
print("Feature:\t\t\t\tImportance:")
for i in indices:
    general_importances += (Summer_L8.feature_metadata.index.values[i], importances[indices[count]])
    if count < 25:
        print(str(count+1)+". "+str(Summer_L8.feature_metadata.index.values[i])+"\t"+str(importances[indices[count]]))
    count += 1
    
general_importances_df = pd.DataFrame(np.array(general_importances).reshape(386,1))

np.savetxt("importances/bone_summer_18S_L8_importances.csv", general_importances_df, delimiter=",", fmt='%s')

Feature:				Importance:
1. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Ascomycota;D_6__Saccharomycotina;D_7__Saccharomycetes	0.06967621303511981
2. D_0__Eukaryota;D_1__Opisthokonta;D_2__Holozoa;D_3__Metazoa (Animalia);D_4__Eumetazoa;D_5__Bilateria;D_6__Nematoda;D_7__Chromadorea	0.06737241830218411
3. D_0__Eukaryota;D_1__Amoebozoa;D_2__Tubulinea;D_3__Euamoebida;D_4__BOLA868;D_5__metagenome;D_6__;D_7__	0.04996081956160955
4. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Ascomycota;D_6__Pezizomycotina;D_7__Eurotiomycetes	0.04209204355254472
5. D_0__Eukaryota;D_1__SAR;D_2__Alveolata;__;__;__;__;__	0.037951770255665915
6. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Ascomycota;D_6__Pezizomycotina;D_7__Sordariomycetes	0.03726767225282134
7. D_0__Eukaryota;D_1__Opisthokonta;D_2__Holozoa;D_3__Metazoa (Animalia);D_4__Eumetazoa;__;__;__	0.03703776707187298
8. D_0__Eukaryota;D_1__Opisthokonta;D_2

## Import rarefied data collapsed at level 7

In [65]:
exp_L7 = ca.read_amplicon('/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/01_qiime2_analysis/03_feature_tables/collapsed_tables/18S_plate52_ribonly_table_full_taxfiltered_214940_L7.biom', '/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/02_metadata/map_18S.txt', min_reads=0, normalize=None)


2020-07-02 14:10:04 WARNING These have metadata but do not have data - dropped (59): {'STAFS.2016.065.L10', 'CSU.extraction.blank11', 'STAFS.2016.065.L11', 'BLANK52.9A', 'BLANK52.9H', 'CSU.extraction.blank1', 'BLANK52.9C', 'BLANK52.10D', 'BLANK52.12A', 'CSU.extraction.blank12', 'CSU.extraction.blank6', 'CSU.extraction.blank4', 'CSU.extraction.blank8', 'CSU.extraction.blank13', 'BLANK52.11D', 'BLANK52.12B', 'BLANK52.9G', 'CSU.extraction.blank10', 'CSU.extraction.blank2', 'BLANK52.11F', 'BLANK52.5B', 'BLANK52.10F', 'BLANK52.11C', 'CSU.extraction.blank14', 'BLANK52.10B', 'SHSU.STAFS2016.011.L09', 'BLANK52.9B', 'STAFS.2016.065.L09', 'BLANK52.10G', 'BLANK52.11A', 'BLANK52.12D', 'BLANK52.10E', 'SHSU.STAFS2016.024.L10', 'CSU.extraction.blank15', 'BLANK52.12F', 'BLANK52.9F', 'BLANK52.11B', 'STAFS.2016.064.R09', 'BLANK52.10A', 'SHSU.STAFS2016.007.L09', 'CSU.extraction.blank9', '#q2:types', 'SHSU.STAFS2016.007.R10', 'BLANK52.11E', 'BLANK52.12G', 'BLANK52.9D', 'BLANK52.11H', 'BLANK52.11G', 'BLANK

In [66]:
#filtering out spring and defining this as "Summer"
Summer_L7 = exp_L7.filter_samples('season', 'summer')
Summer_L7.sample_metadata.host_subject_id.value_counts()

STAFS2016.064    7
STAFS2016.067    6
STAFS2016.065    5
Name: host_subject_id, dtype: int64

In [67]:
print(Summer_L7.feature_metadata)

                                                                                          _feature_id
D_0__Eukaryota;D_1__Amoebozoa;D_2__Cavosteliida...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Cavosteliid...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Dictyostelia...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Dictyosteli...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;

## Running the L7 model

In [68]:
# groupKfold = 3, will leave out one body for each model
gkf = GroupKFold(3)

X = Summer_L7.data
y = Summer_L7.sample_metadata['ADD']
y = (y.astype(float))

groups = Summer_L7.sample_metadata['host_subject_id']

# used to test the param grid for parameter tuning
# use the output of this estimator below for input into new estimator (not commented out)
param_grid = {"max_depth": [4, 8, 16, None],
              "max_features": ['sqrt', 'log2', 0.1],
              "min_samples_split": [0.001, 0.01, 0.1],
              "min_weight_fraction_leaf": [0.0001, 0.001, 0.01],
              "bootstrap": [True, False]}

#param_grid = {"max_depth": [8],
#          "max_features": [0.1],
#          "min_samples_split": [0.001],
#          "min_weight_fraction_leaf": [0.0001],
#          "bootstrap": [False]}

rf = RandomForestRegressor(n_estimators=1000, random_state=999, criterion='mae')
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=1)

In [69]:
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x11fb30ed0>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mae', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_es...
                                             oob_score=False, random_state=999,
                                             verbose=0, warm_start=False

In [70]:
# this line is used when run with the first param grid to determine what the best parameters are for tuning
print(gs.best_params_)

{'bootstrap': False, 'max_depth': 16, 'max_features': 'sqrt', 'min_samples_split': 0.001, 'min_weight_fraction_leaf': 0.0001}


In [71]:
print('The best mean absolute error is', round(-gs.best_score_,2))

The best mean absolute error is 820.67


In [72]:
joblib.dump(gs.best_estimator_, 'models/bone_summer_18S_L7.pkl')

['models/bone_summer_18S_L7.pkl']

In [73]:
bone_summer_L7 = joblib.load('models/bone_summer_18S_L7.pkl')
bone_summer_L7

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=16, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.001, min_weight_fraction_leaf=0.0001,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

## Determine important features of the L7 model

In [74]:
importances = bone_summer_L7.feature_importances_
std = np.std([tree.feature_importances_ for tree in bone_summer_L7.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

feature_metadata = Summer_L7.feature_metadata
np.savetxt("importances/bone_summer_18S_L7_features.csv", feature_metadata, delimiter=",", fmt='%s')


#print the important ids in order
general_importances = []
count = 0
print("Feature:\t\t\t\tImportance:")
for i in indices:
    general_importances += (Summer_L7.feature_metadata.index.values[i], importances[indices[count]])
    if count < 25:
        print(str(count+1)+". "+str(Summer_L7.feature_metadata.index.values[i])+"\t"+str(importances[indices[count]]))
    count += 1
    
general_importances_df = pd.DataFrame(np.array(general_importances).reshape(288,1))

np.savetxt("importances/bone_summer_18S_L7_importances.csv", general_importances_df, delimiter=",", fmt='%s')

Feature:				Importance:
1. D_0__Eukaryota;D_1__Opisthokonta;D_2__Holozoa;D_3__Metazoa (Animalia);D_4__Eumetazoa;D_5__Bilateria;D_6__Nematoda	0.07145102038339576
2. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Ascomycota;D_6__Saccharomycotina	0.05807602171934425
3. D_0__Eukaryota;D_1__Amoebozoa;D_2__Tubulinea;D_3__Euamoebida;D_4__BOLA868;D_5__metagenome;D_6__	0.05645498663118672
4. D_0__Eukaryota;D_1__SAR;D_2__Alveolata;__;__;__;__	0.04136638355945472
5. D_0__Eukaryota;D_1__Opisthokonta;D_2__Holozoa;D_3__Metazoa (Animalia);D_4__Eumetazoa;__;__	0.036680788439407495
6. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Basidiomycota;D_6__Agaricomycotina	0.0362463534358252
7. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Ascomycota;D_6__Pezizomycotina	0.036087014973071936
8. D_0__Eukaryota;D_1__SAR;D_2__Rhizaria;D_3__Cercozoa;D_4__Glissomonadida;D_5__Heteromita;D_6__metagenome	0.03080315964239488

## Import rarefied data collapsed at level 6

In [75]:
exp_L6 = ca.read_amplicon('/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/01_qiime2_analysis/03_feature_tables/collapsed_tables/18S_plate52_ribonly_table_full_taxfiltered_214940_L6.biom', '/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/02_metadata/map_18S.txt', min_reads=0, normalize=None)


2020-07-02 14:21:14 WARNING These have metadata but do not have data - dropped (59): {'STAFS.2016.065.L10', 'CSU.extraction.blank11', 'STAFS.2016.065.L11', 'BLANK52.9A', 'BLANK52.9H', 'CSU.extraction.blank1', 'BLANK52.9C', 'BLANK52.10D', 'BLANK52.12A', 'CSU.extraction.blank12', 'CSU.extraction.blank6', 'CSU.extraction.blank4', 'CSU.extraction.blank8', 'CSU.extraction.blank13', 'BLANK52.11D', 'BLANK52.12B', 'BLANK52.9G', 'CSU.extraction.blank10', 'CSU.extraction.blank2', 'BLANK52.11F', 'BLANK52.5B', 'BLANK52.10F', 'BLANK52.11C', 'CSU.extraction.blank14', 'BLANK52.10B', 'SHSU.STAFS2016.011.L09', 'BLANK52.9B', 'STAFS.2016.065.L09', 'BLANK52.10G', 'BLANK52.11A', 'BLANK52.12D', 'BLANK52.10E', 'SHSU.STAFS2016.024.L10', 'CSU.extraction.blank15', 'BLANK52.12F', 'BLANK52.9F', 'BLANK52.11B', 'STAFS.2016.064.R09', 'BLANK52.10A', 'SHSU.STAFS2016.007.L09', 'CSU.extraction.blank9', '#q2:types', 'SHSU.STAFS2016.007.R10', 'BLANK52.11E', 'BLANK52.12G', 'BLANK52.9D', 'BLANK52.11H', 'BLANK52.11G', 'BLANK

In [76]:
#filtering out spring and defining this as "Summer"
Summer_L6 = exp_L6.filter_samples('season', 'summer')
Summer_L6.sample_metadata.host_subject_id.value_counts()

STAFS2016.064    7
STAFS2016.067    6
STAFS2016.065    5
Name: host_subject_id, dtype: int64

In [77]:
print(Summer_L6.feature_metadata)

                                                                                          _feature_id
D_0__Eukaryota;D_1__Amoebozoa;D_2__Cavosteliida...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Cavosteliid...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Dictyostelia...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Dictyosteli...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Gracilipodid...  D_0__Eukaryota;D_1__Amoebozoa;

## Running the L6 model

In [78]:
# groupKfold = 3, will leave out one body for each model
gkf = GroupKFold(3)

X = Summer_L6.data
y = Summer_L6.sample_metadata['ADD']
y = (y.astype(float))

groups = Summer_L6.sample_metadata['host_subject_id']

# used to test the param grid for parameter tuning
# use the output of this estimator below for input into new estimator (not commented out)
param_grid = {"max_depth": [4, 8, 16, None],
              "max_features": ['sqrt', 'log2', 0.1],
              "min_samples_split": [0.001, 0.01, 0.1],
              "min_weight_fraction_leaf": [0.0001, 0.001, 0.01],
              "bootstrap": [True, False]}

#param_grid = {"max_depth": [8],
#          "max_features": [0.1],
#          "min_samples_split": [0.001],
#          "min_weight_fraction_leaf": [0.0001],
#          "bootstrap": [False]}

rf = RandomForestRegressor(n_estimators=1000, random_state=999, criterion='mae')
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=1)

In [79]:
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x11ff49750>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mae', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_es...
                                             oob_score=False, random_state=999,
                                             verbose=0, warm_start=False

In [80]:
# this line is used when run with the first param grid to determine what the best parameters are for tuning
print(gs.best_params_)

{'bootstrap': False, 'max_depth': 16, 'max_features': 'sqrt', 'min_samples_split': 0.001, 'min_weight_fraction_leaf': 0.0001}


In [81]:
print('The best mean absolute error is', round(-gs.best_score_,2))

The best mean absolute error is 867.09


In [82]:
joblib.dump(gs.best_estimator_, 'models/bone_summer_18S_L6.pkl')

['models/bone_summer_18S_L6.pkl']

In [83]:
bone_summer_L6 = joblib.load('models/bone_summer_18S_L6.pkl')
bone_summer_L6

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=16, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.001, min_weight_fraction_leaf=0.0001,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

## Determine important features of the L6 model

In [84]:
importances = bone_summer_L6.feature_importances_
std = np.std([tree.feature_importances_ for tree in bone_summer_L6.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

feature_metadata = Summer_L6.feature_metadata
np.savetxt("importances/bone_summer_18S_L6_features.csv", feature_metadata, delimiter=",", fmt='%s')


#print the important ids in order
general_importances = []
count = 0
print("Feature:\t\t\t\tImportance:")
for i in indices:
    general_importances += (Summer_L6.feature_metadata.index.values[i], importances[indices[count]])
    if count < 25:
        print(str(count+1)+". "+str(Summer_L6.feature_metadata.index.values[i])+"\t"+str(importances[indices[count]]))
    count += 1
    
general_importances_df = pd.DataFrame(np.array(general_importances).reshape(220,1))

np.savetxt("importances/bone_summer_18S_L6_importances.csv", general_importances_df, delimiter=",", fmt='%s')

Feature:				Importance:
1. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Ascomycota	0.07153030819565671
2. D_0__Eukaryota;D_1__Opisthokonta;D_2__Holozoa;D_3__Metazoa (Animalia);D_4__Eumetazoa;D_5__Bilateria	0.06485042265506961
3. D_0__Eukaryota;D_1__Amoebozoa;D_2__Tubulinea;D_3__Euamoebida;D_4__BOLA868;D_5__metagenome	0.05266154437945154
4. D_0__Eukaryota;D_1__Opisthokonta;D_2__Holozoa;D_3__Metazoa (Animalia);D_4__Eumetazoa;__	0.050277886264787255
5. D_0__Eukaryota;D_1__SAR;D_2__Alveolata;__;__;__	0.04846375674463879
6. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya;D_5__Basidiomycota	0.04248318626640372
7. D_0__Eukaryota;D_1__Opisthokonta;D_2__Holozoa;__;__;__	0.036306785159463874
8. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;__;__	0.03507682000424404
9. D_0__Eukaryota;D_1__SAR;D_2__Alveolata;D_3__Apicomplexa;D_4__Conoidasida;D_5__Gregarinasina	0.03428278317233025
10. D_0__Eukaryota;__;__;__;__;__	0.03115495

## Import rarefied data collapsed at level 5

In [85]:
exp_L5 = ca.read_amplicon('/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/01_qiime2_analysis/03_feature_tables/collapsed_tables/18S_plate52_ribonly_table_full_taxfiltered_214940_L5.biom', '/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/02_metadata/map_18S.txt', min_reads=0, normalize=None)


2020-07-02 14:32:23 WARNING These have metadata but do not have data - dropped (59): {'STAFS.2016.065.L10', 'CSU.extraction.blank11', 'STAFS.2016.065.L11', 'BLANK52.9A', 'BLANK52.9H', 'CSU.extraction.blank1', 'BLANK52.9C', 'BLANK52.10D', 'BLANK52.12A', 'CSU.extraction.blank12', 'CSU.extraction.blank6', 'CSU.extraction.blank4', 'CSU.extraction.blank8', 'CSU.extraction.blank13', 'BLANK52.11D', 'BLANK52.12B', 'BLANK52.9G', 'CSU.extraction.blank10', 'CSU.extraction.blank2', 'BLANK52.11F', 'BLANK52.5B', 'BLANK52.10F', 'BLANK52.11C', 'CSU.extraction.blank14', 'BLANK52.10B', 'SHSU.STAFS2016.011.L09', 'BLANK52.9B', 'STAFS.2016.065.L09', 'BLANK52.10G', 'BLANK52.11A', 'BLANK52.12D', 'BLANK52.10E', 'SHSU.STAFS2016.024.L10', 'CSU.extraction.blank15', 'BLANK52.12F', 'BLANK52.9F', 'BLANK52.11B', 'STAFS.2016.064.R09', 'BLANK52.10A', 'SHSU.STAFS2016.007.L09', 'CSU.extraction.blank9', '#q2:types', 'SHSU.STAFS2016.007.R10', 'BLANK52.11E', 'BLANK52.12G', 'BLANK52.9D', 'BLANK52.11H', 'BLANK52.11G', 'BLANK

In [86]:
#filtering out spring and defining this as "Summer"
Summer_L5 = exp_L5.filter_samples('season', 'summer')
Summer_L5.sample_metadata.host_subject_id.value_counts()

STAFS2016.064    7
STAFS2016.067    6
STAFS2016.065    5
Name: host_subject_id, dtype: int64

In [87]:
print(Summer_L5.feature_metadata)

                                                                                          _feature_id
D_0__Eukaryota;D_1__Amoebozoa;D_2__Cavosteliida...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Cavosteliid...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Dictyostelia...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Dictyosteli...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Gracilipodid...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Gracilipodi...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Incertae Sed...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Incertae Se...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Mycamoeba;D_...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Mycamoeba;D...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Myxogastria;...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Myxogastria...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Myxogastria;...  D_0__Eukaryota;D_1__Amoebozoa;

## Running the L5 model

In [88]:
# groupKfold = 3, will leave out one body for each model
gkf = GroupKFold(3)

X = Summer_L5.data
y = Summer_L5.sample_metadata['ADD']
y = (y.astype(float))

groups = Summer_L5.sample_metadata['host_subject_id']

# used to test the param grid for parameter tuning
# use the output of this estimator below for input into new estimator (not commented out)
param_grid = {"max_depth": [4, 8, 16, None],
              "max_features": ['sqrt', 'log2', 0.1],
              "min_samples_split": [0.001, 0.01, 0.1],
              "min_weight_fraction_leaf": [0.0001, 0.001, 0.01],
              "bootstrap": [True, False]}

#param_grid = {"max_depth": [8],
#          "max_features": [0.1],
#          "min_samples_split": [0.001],
#          "min_weight_fraction_leaf": [0.0001],
#          "bootstrap": [False]}

rf = RandomForestRegressor(n_estimators=1000, random_state=999, criterion='mae')
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=1)

In [89]:
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x1201128d0>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mae', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_es...
                                             oob_score=False, random_state=999,
                                             verbose=0, warm_start=False

In [90]:
# this line is used when run with the first param grid to determine what the best parameters are for tuning
print(gs.best_params_)

{'bootstrap': False, 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_split': 0.001, 'min_weight_fraction_leaf': 0.0001}


In [91]:
print('The best mean absolute error is', round(-gs.best_score_,2))

The best mean absolute error is 942.52


In [92]:
joblib.dump(gs.best_estimator_, 'models/bone_summer_18S_L5.pkl')

['models/bone_summer_18S_L5.pkl']

In [93]:
bone_summer_L5 = joblib.load('models/bone_summer_18S_L5.pkl')
bone_summer_L5

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=8, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.001, min_weight_fraction_leaf=0.0001,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

## Determine important features of the L5 model

In [94]:
importances = bone_summer_L5.feature_importances_
std = np.std([tree.feature_importances_ for tree in bone_summer_L5.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

feature_metadata = Summer_L5.feature_metadata
np.savetxt("importances/bone_summer_18S_L5_features.csv", feature_metadata, delimiter=",", fmt='%s')


#print the important ids in order
general_importances = []
count = 0
print("Feature:\t\t\t\tImportance:")
for i in indices:
    general_importances += (Summer_L5.feature_metadata.index.values[i], importances[indices[count]])
    if count < 25:
        print(str(count+1)+". "+str(Summer_L5.feature_metadata.index.values[i])+"\t"+str(importances[indices[count]]))
    count += 1
    
general_importances_df = pd.DataFrame(np.array(general_importances).reshape(136,1))

np.savetxt("importances/bone_summer_18S_L5_importances.csv", general_importances_df, delimiter=",", fmt='%s')

Feature:				Importance:
1. D_0__Eukaryota;D_1__Opisthokonta;D_2__Holozoa;D_3__Metazoa (Animalia);D_4__Eumetazoa	0.09415741764882177
2. D_0__Eukaryota;D_1__Amoebozoa;D_2__Tubulinea;D_3__Euamoebida;D_4__BOLA868	0.08136066049455005
3. D_0__Eukaryota;D_1__SAR;D_2__Alveolata;__;__	0.062262808971088136
4. D_0__Eukaryota;D_1__SAR;D_2__Alveolata;D_3__Apicomplexa;D_4__Conoidasida	0.056138941313990956
5. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;__	0.055582179692787594
6. D_0__Eukaryota;D_1__Opisthokonta;D_2__Holozoa;__;__	0.05376410814647099
7. D_0__Eukaryota;__;__;__;__	0.049980436600682125
8. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Dikarya	0.04418111962815736
9. D_0__Eukaryota;D_1__Opisthokonta;D_2__Holozoa;D_3__Metazoa (Animalia);__	0.035237228721509514
10. D_0__Eukaryota;D_1__SAR;D_2__Stramenopiles;D_3__Labyrinthulomycetes;D_4__Sorodiplophrys	0.028755168415253277
11. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi;D_4__Zoopagomycota	

## Import rarefied data collapsed at level 4

In [95]:
exp_L4 = ca.read_amplicon('/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/01_qiime2_analysis/03_feature_tables/collapsed_tables/18S_plate52_ribonly_table_full_taxfiltered_214940_L4.biom', '/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/02_metadata/map_18S.txt', min_reads=0, normalize=None)


2020-07-02 14:43:16 WARNING These have metadata but do not have data - dropped (59): {'STAFS.2016.065.L10', 'CSU.extraction.blank11', 'STAFS.2016.065.L11', 'BLANK52.9A', 'BLANK52.9H', 'CSU.extraction.blank1', 'BLANK52.9C', 'BLANK52.10D', 'BLANK52.12A', 'CSU.extraction.blank12', 'CSU.extraction.blank6', 'CSU.extraction.blank4', 'CSU.extraction.blank8', 'CSU.extraction.blank13', 'BLANK52.11D', 'BLANK52.12B', 'BLANK52.9G', 'CSU.extraction.blank10', 'CSU.extraction.blank2', 'BLANK52.11F', 'BLANK52.5B', 'BLANK52.10F', 'BLANK52.11C', 'CSU.extraction.blank14', 'BLANK52.10B', 'SHSU.STAFS2016.011.L09', 'BLANK52.9B', 'STAFS.2016.065.L09', 'BLANK52.10G', 'BLANK52.11A', 'BLANK52.12D', 'BLANK52.10E', 'SHSU.STAFS2016.024.L10', 'CSU.extraction.blank15', 'BLANK52.12F', 'BLANK52.9F', 'BLANK52.11B', 'STAFS.2016.064.R09', 'BLANK52.10A', 'SHSU.STAFS2016.007.L09', 'CSU.extraction.blank9', '#q2:types', 'SHSU.STAFS2016.007.R10', 'BLANK52.11E', 'BLANK52.12G', 'BLANK52.9D', 'BLANK52.11H', 'BLANK52.11G', 'BLANK

In [96]:
#filtering out spring and defining this as "Summer"
Summer_L4 = exp_L4.filter_samples('season', 'summer')
Summer_L4.sample_metadata.host_subject_id.value_counts()

STAFS2016.064    7
STAFS2016.067    6
STAFS2016.065    5
Name: host_subject_id, dtype: int64

In [97]:
print(Summer_L4.feature_metadata)

                                                                                          _feature_id
D_0__Eukaryota;D_1__Amoebozoa;D_2__Cavosteliida...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Cavosteliid...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Dictyostelia...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Dictyosteli...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Gracilipodid...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Gracilipodi...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Incertae Sed...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Incertae Se...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Mycamoeba;D_...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Mycamoeba;D...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Myxogastria;...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Myxogastria...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Myxogastria;...  D_0__Eukaryota;D_1__Amoebozoa;

## Running the L4 model

In [98]:
# groupKfold = 3, will leave out one body for each model
gkf = GroupKFold(3)

X = Summer_L4.data
y = Summer_L4.sample_metadata['ADD']
y = (y.astype(float))

groups = Summer_L4.sample_metadata['host_subject_id']

# used to test the param grid for parameter tuning
# use the output of this estimator below for input into new estimator (not commented out)
param_grid = {"max_depth": [4, 8, 16, None],
              "max_features": ['sqrt', 'log2', 0.1],
              "min_samples_split": [0.001, 0.01, 0.1],
              "min_weight_fraction_leaf": [0.0001, 0.001, 0.01],
              "bootstrap": [True, False]}

#param_grid = {"max_depth": [8],
#          "max_features": [0.1],
#          "min_samples_split": [0.001],
#          "min_weight_fraction_leaf": [0.0001],
#          "bootstrap": [False]}

rf = RandomForestRegressor(n_estimators=1000, random_state=999, criterion='mae')
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=1)

In [99]:
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x11fb1d2d0>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mae', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_es...
                                             oob_score=False, random_state=999,
                                             verbose=0, warm_start=False

In [100]:
# this line is used when run with the first param grid to determine what the best parameters are for tuning
print(gs.best_params_)

{'bootstrap': False, 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_split': 0.001, 'min_weight_fraction_leaf': 0.0001}


In [101]:
print('The best mean absolute error is', round(-gs.best_score_,2))

The best mean absolute error is 904.18


In [102]:
joblib.dump(gs.best_estimator_, 'models/bone_summer_18S_L4.pkl')

['models/bone_summer_18S_L4.pkl']

In [103]:
bone_summer_L4 = joblib.load('models/bone_summer_18S_L4.pkl')
bone_summer_L4

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=8, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.001, min_weight_fraction_leaf=0.0001,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

## Determine important features of the L4 model

In [104]:
importances = bone_summer_L4.feature_importances_
std = np.std([tree.feature_importances_ for tree in bone_summer_L4.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

feature_metadata = Summer_L4.feature_metadata
np.savetxt("importances/bone_summer_18S_L4_features.csv", feature_metadata, delimiter=",", fmt='%s')


#print the important ids in order
general_importances = []
count = 0
print("Feature:\t\t\t\tImportance:")
for i in indices:
    general_importances += (Summer_L4.feature_metadata.index.values[i], importances[indices[count]])
    if count < 25:
        print(str(count+1)+". "+str(Summer_L4.feature_metadata.index.values[i])+"\t"+str(importances[indices[count]]))
    count += 1
    
general_importances_df = pd.DataFrame(np.array(general_importances).reshape(80,1))

np.savetxt("importances/bone_summer_18S_L4_importances.csv", general_importances_df, delimiter=",", fmt='%s')

Feature:				Importance:
1. D_0__Eukaryota;D_1__Opisthokonta;D_2__Holozoa;D_3__Metazoa (Animalia)	0.11537907510790993
2. D_0__Eukaryota;D_1__Amoebozoa;D_2__Tubulinea;D_3__Euamoebida	0.10879095257152173
3. D_0__Eukaryota;D_1__SAR;D_2__Alveolata;__	0.0778651285501217
4. D_0__Eukaryota;D_1__SAR;D_2__Alveolata;D_3__Apicomplexa	0.07110421496753404
5. D_0__Eukaryota;__;__;__	0.06770191759458177
6. D_0__Eukaryota;D_1__Opisthokonta;D_2__Holozoa;__	0.06276389843555816
7. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea;D_3__Fungi	0.060727734797413904
8. D_0__Eukaryota;D_1__SAR;D_2__Stramenopiles;D_3__Labyrinthulomycetes	0.04096948302876602
9. D_0__Eukaryota;D_1__Opisthokonta;__;__	0.03504670431874672
10. D_0__Eukaryota;D_1__SAR;D_2__Rhizaria;D_3__Cercozoa	0.02924095499736783
11. D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea;D_3__Longamoebia	0.02763083631888977
12. D_0__Eukaryota;D_1__SAR;D_2__Alveolata;D_3__Ciliophora	0.02691121768356815
13. D_0__Eukaryota;D_1__SAR;D_2__Stramenopiles;D_3__Bicosoe

## Import rarefied data collapsed at level 3

In [105]:
exp_L3 = ca.read_amplicon('/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/01_qiime2_analysis/03_feature_tables/collapsed_tables/18S_plate52_ribonly_table_full_taxfiltered_214940_L3.biom', '/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/02_metadata/map_18S.txt', min_reads=0, normalize=None)


2020-07-02 14:53:45 WARNING These have metadata but do not have data - dropped (59): {'STAFS.2016.065.L10', 'CSU.extraction.blank11', 'STAFS.2016.065.L11', 'BLANK52.9A', 'BLANK52.9H', 'CSU.extraction.blank1', 'BLANK52.9C', 'BLANK52.10D', 'BLANK52.12A', 'CSU.extraction.blank12', 'CSU.extraction.blank6', 'CSU.extraction.blank4', 'CSU.extraction.blank8', 'CSU.extraction.blank13', 'BLANK52.11D', 'BLANK52.12B', 'BLANK52.9G', 'CSU.extraction.blank10', 'CSU.extraction.blank2', 'BLANK52.11F', 'BLANK52.5B', 'BLANK52.10F', 'BLANK52.11C', 'CSU.extraction.blank14', 'BLANK52.10B', 'SHSU.STAFS2016.011.L09', 'BLANK52.9B', 'STAFS.2016.065.L09', 'BLANK52.10G', 'BLANK52.11A', 'BLANK52.12D', 'BLANK52.10E', 'SHSU.STAFS2016.024.L10', 'CSU.extraction.blank15', 'BLANK52.12F', 'BLANK52.9F', 'BLANK52.11B', 'STAFS.2016.064.R09', 'BLANK52.10A', 'SHSU.STAFS2016.007.L09', 'CSU.extraction.blank9', '#q2:types', 'SHSU.STAFS2016.007.R10', 'BLANK52.11E', 'BLANK52.12G', 'BLANK52.9D', 'BLANK52.11H', 'BLANK52.11G', 'BLANK

In [106]:
#filtering out spring and defining this as "Summer"
Summer_L3 = exp_L3.filter_samples('season', 'summer')
Summer_L3.sample_metadata.host_subject_id.value_counts()

STAFS2016.064    7
STAFS2016.067    6
STAFS2016.065    5
Name: host_subject_id, dtype: int64

In [107]:
print(Summer_L3.feature_metadata)

                                                                                          _feature_id
D_0__Eukaryota;D_1__Amoebozoa;D_2__Cavosteliida       D_0__Eukaryota;D_1__Amoebozoa;D_2__Cavosteliida
D_0__Eukaryota;D_1__Amoebozoa;D_2__Dictyostelia       D_0__Eukaryota;D_1__Amoebozoa;D_2__Dictyostelia
D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea               D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea
D_0__Eukaryota;D_1__Amoebozoa;D_2__Gracilipodida     D_0__Eukaryota;D_1__Amoebozoa;D_2__Gracilipodida
D_0__Eukaryota;D_1__Amoebozoa;D_2__Incertae Sedis   D_0__Eukaryota;D_1__Amoebozoa;D_2__Incertae Sedis
D_0__Eukaryota;D_1__Amoebozoa;D_2__Mycamoeba             D_0__Eukaryota;D_1__Amoebozoa;D_2__Mycamoeba
D_0__Eukaryota;D_1__Amoebozoa;D_2__Myxogastria         D_0__Eukaryota;D_1__Amoebozoa;D_2__Myxogastria
D_0__Eukaryota;D_1__Amoebozoa;D_2__Schizoplasmo...  D_0__Eukaryota;D_1__Amoebozoa;D_2__Schizoplasm...
D_0__Eukaryota;D_1__Amoebozoa;D_2__Tubulinea             D_0__Eukaryota;D_1__Amoeb

## Running the L3 model

In [108]:
# groupKfold = 3, will leave out one body for each model
gkf = GroupKFold(3)

X = Summer_L3.data
y = Summer_L3.sample_metadata['ADD']
y = (y.astype(float))

groups = Summer_L3.sample_metadata['host_subject_id']

# used to test the param grid for parameter tuning
# use the output of this estimator below for input into new estimator (not commented out)
param_grid = {"max_depth": [4, 8, 16, None],
              "max_features": ['sqrt', 'log2', 0.1],
              "min_samples_split": [0.001, 0.01, 0.1],
              "min_weight_fraction_leaf": [0.0001, 0.001, 0.01],
              "bootstrap": [True, False]}

#param_grid = {"max_depth": [8],
#          "max_features": [0.1],
#          "min_samples_split": [0.001],
#          "min_weight_fraction_leaf": [0.0001],
#          "bootstrap": [False]}

rf = RandomForestRegressor(n_estimators=1000, random_state=999, criterion='mae')
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=1)

In [109]:
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x11fd15550>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mae', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_es...
                                             oob_score=False, random_state=999,
                                             verbose=0, warm_start=False

In [110]:
# this line is used when run with the first param grid to determine what the best parameters are for tuning
print(gs.best_params_)

{'bootstrap': False, 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_split': 0.001, 'min_weight_fraction_leaf': 0.0001}


In [111]:
print('The best mean absolute error is', round(-gs.best_score_,2))

The best mean absolute error is 958.75


In [112]:
joblib.dump(gs.best_estimator_, 'models/bone_summer_18S_L3.pkl')

['models/bone_summer_18S_L3.pkl']

In [113]:
bone_summer_L3 = joblib.load('models/bone_summer_18S_L3.pkl')
bone_summer_L3

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=8, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.001, min_weight_fraction_leaf=0.0001,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

## Determine important features of the L3 model

In [114]:
importances = bone_summer_L3.feature_importances_
std = np.std([tree.feature_importances_ for tree in bone_summer_L3.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

feature_metadata = Summer_L3.feature_metadata
np.savetxt("importances/bone_summer_18S_L3_features.csv", feature_metadata, delimiter=",", fmt='%s')


#print the important ids in order
general_importances = []
count = 0
print("Feature:\t\t\t\tImportance:")
for i in indices:
    general_importances += (Summer_L3.feature_metadata.index.values[i], importances[indices[count]])
    if count < 25:
        print(str(count+1)+". "+str(Summer_L3.feature_metadata.index.values[i])+"\t"+str(importances[indices[count]]))
    count += 1
    
general_importances_df = pd.DataFrame(np.array(general_importances).reshape(44,1))

np.savetxt("importances/bone_summer_18S_L3_importances.csv", general_importances_df, delimiter=",", fmt='%s')

Feature:				Importance:
1. D_0__Eukaryota;D_1__Opisthokonta;D_2__Holozoa	0.17097204100650593
2. D_0__Eukaryota;D_1__Amoebozoa;D_2__Tubulinea	0.16436830175254882
3. D_0__Eukaryota;__;__	0.11093485719085554
4. D_0__Eukaryota;D_1__Opisthokonta;D_2__Nucletmycea	0.10762872228667412
5. D_0__Eukaryota;D_1__SAR;D_2__Stramenopiles	0.09699542140590808
6. D_0__Eukaryota;D_1__SAR;D_2__Alveolata	0.06941753894983607
7. D_0__Eukaryota;D_1__Opisthokonta;__	0.05711401318925566
8. D_0__Eukaryota;D_1__Excavata;D_2__Discoba	0.04315143803713639
9. D_0__Eukaryota;D_1__SAR;D_2__Rhizaria	0.04180333984959237
10. D_0__Eukaryota;D_1__Amoebozoa;D_2__Discosea	0.03919230069184913
11. D_0__Eukaryota;D_1__Amoebozoa;D_2__Dictyostelia	0.020461864877141193
12. D_0__Eukaryota;D_1__Amoebozoa;D_2__uncultured	0.010992990735539094
13. D_0__Eukaryota;D_1__Amoebozoa;D_2__Myxogastria	0.010789421387289416
14. D_0__Eukaryota;D_1__Amoebozoa;D_2__Mycamoeba	0.010395780107393232
15. D_0__Eukaryota;D_1__Amoebozoa;D_2__Schizoplasmodiid

## Import rarefied data collapsed at level 2

In [3]:
exp_L2 = ca.read_amplicon('/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/01_qiime2_analysis/03_feature_tables/collapsed_tables/18S_plate52_ribonly_table_full_taxfiltered_214940_L2.biom', '/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/02_18S/Hiseq_original_run/forward_reads_only/02_metadata/map_18S.txt', min_reads=0, normalize=None)


2021-06-01 14:02:06 WARNING These have metadata but do not have data - dropped (59): {'CSU.extraction.blank7', 'BLANK52.12A', 'BLANK52.11H', 'CSU.extraction.blank10', 'BLANK52.10B', 'BLANK52.12F', 'BLANK52.9D', 'BLANK52.10E', 'STAFS.2016.065.L11', 'BLANK52.9B', 'SHSU.STAFS2016.007.R10', 'CSU.extraction.blank13', 'BLANK52.12D', 'BLANK52.11A', 'BLANK52.9C', 'BLANK52.9G', 'BLANK52.10C', 'CSU.extraction.blank4', 'BLANK52.12C', 'BLANK52.11G', 'BLANK52.9A', 'BLANK52.11B', 'STAFS.2016.067.R12', 'CSU.extraction.blank11', 'BLANK52.9E', 'CSU.extraction.blank9', 'CSU.extraction.blank6', 'STAFS.2016.067.L11.march', 'CSU.extraction.blank12', 'BLANK52.10A', 'CSU.extraction.blank15', 'BLANK52.9H', 'BLANK52.11C', 'BLANK52.10D', '#q2:types', 'SHSU.STAFS2016.024.L10', 'BLANK52.12H', 'BLANK52.11E', 'STAFS.2016.064.R09', 'CSU.extraction.blank3', 'CSU.extraction.blank14', 'SHSU.STAFS2016.007.L09', 'STAFS.2016.065.L10', 'BLANK52.10G', 'SHSU.STAFS2016.011.L09', 'BLANK52.9F', 'BLANK52.11D', 'CSU.extraction.bl

In [4]:
#filtering out spring and defining this as "Summer"
Summer_L2 = exp_L2.filter_samples('season', 'summer')
Summer_L2.sample_metadata.host_subject_id.value_counts()

STAFS2016.064    7
STAFS2016.067    6
STAFS2016.065    5
Name: host_subject_id, dtype: int64

In [5]:
print(Summer_L2.feature_metadata)

                                                         _feature_id
D_0__Eukaryota;D_1__Amoebozoa          D_0__Eukaryota;D_1__Amoebozoa
D_0__Eukaryota;D_1__Cryptophyceae  D_0__Eukaryota;D_1__Cryptophyceae
D_0__Eukaryota;D_1__Excavata            D_0__Eukaryota;D_1__Excavata
D_0__Eukaryota;D_1__Opisthokonta    D_0__Eukaryota;D_1__Opisthokonta
D_0__Eukaryota;D_1__SAR                      D_0__Eukaryota;D_1__SAR
D_0__Eukaryota;__                                  D_0__Eukaryota;__


## Running the L2 model

In [118]:
# groupKfold = 3, will leave out one body for each model
gkf = GroupKFold(3)

X = Summer_L2.data
y = Summer_L2.sample_metadata['ADD']
y = (y.astype(float))

groups = Summer_L2.sample_metadata['host_subject_id']

# used to test the param grid for parameter tuning
# use the output of this estimator below for input into new estimator (not commented out)
#param_grid = {"max_depth": [4, 8, 16, None],
#              "max_features": ['sqrt', 'log2', 0.1],
#              "min_samples_split": [0.001, 0.01, 0.1],
              "min_weight_fraction_leaf": [0.0001, 0.001, 0.01],
              "bootstrap": [True, False]}

#param_grid = {"max_depth": [16],
#          "max_features": [0.1],
#          "min_samples_split": [0.001],
#          "min_weight_fraction_leaf": [0.0001],
#          "bootstrap": [False]}

rf = RandomForestRegressor(n_estimators=1000, random_state=999, criterion='mae')
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=1)

In [119]:
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x1201509d0>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mae', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_es...
                                             oob_score=False, random_state=999,
                                             verbose=0, warm_start=False

In [120]:
# this line is used when run with the first param grid to determine what the best parameters are for tuning
print(gs.best_params_)

{'bootstrap': True, 'max_depth': 16, 'max_features': 0.1, 'min_samples_split': 0.001, 'min_weight_fraction_leaf': 0.0001}


In [121]:
print('The best mean absolute error is', round(-gs.best_score_,2))

The best mean absolute error is 1050.45


In [122]:
joblib.dump(gs.best_estimator_, 'models/bone_summer_18S_L2.pkl')

['models/bone_summer_18S_L2.pkl']

In [123]:
bone_summer_L2 = joblib.load('models/bone_summer_18S_L2.pkl')
bone_summer_L2

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mae',
                      max_depth=16, max_features=0.1, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.001, min_weight_fraction_leaf=0.0001,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

## Determine important features of the L2 model

In [124]:
importances = bone_summer_L2.feature_importances_
std = np.std([tree.feature_importances_ for tree in bone_summer_L2.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

feature_metadata = Summer_L2.feature_metadata
np.savetxt("importances/bone_summer_18S_L2_features.csv", feature_metadata, delimiter=",", fmt='%s')


#print the important ids in order
general_importances = []
count = 0
print("Feature:\t\t\t\tImportance:")
for i in indices:
    general_importances += (Summer_L2.feature_metadata.index.values[i], importances[indices[count]])
    if count < 25:
        print(str(count+1)+". "+str(Summer_L2.feature_metadata.index.values[i])+"\t"+str(importances[indices[count]]))
    count += 1
    
general_importances_df = pd.DataFrame(np.array(general_importances).reshape(12,1))

np.savetxt("importances/bone_summer_18S_L2_importances.csv", general_importances_df, delimiter=",", fmt='%s')

Feature:				Importance:
1. D_0__Eukaryota;D_1__Amoebozoa	0.23473075537863164
2. D_0__Eukaryota;__	0.2020220631843998
3. D_0__Eukaryota;D_1__Excavata	0.19876165775846652
4. D_0__Eukaryota;D_1__SAR	0.18695092525057236
5. D_0__Eukaryota;D_1__Opisthokonta	0.17753459842792962
6. D_0__Eukaryota;D_1__Cryptophyceae	0.0
